# Plan: OpenVaccine (stanford-covid-vaccine) — Medal-Oriented Workflow

Objectives:
- Build a strong baseline fast, align CV with LB, iterate to medal.

Datasets:
- train.json, test.json, sample_submission.csv

Metric:
- multi-class-log-loss (per-position targets across 5 tasks: reactivity, deg_Mg_pH10, deg_pH10, deg_Mg_50C, deg_50C for L=107 or 130 seq lengths depending on data).

High-level Strategy:
1) Environment & GPU:
- Verify GPU with nvidia-smi and torch CUDA stack sanity.

2) Data Audit:
- Load train.json/test.json; inspect schema, lengths, missing values, target columns, sequence length distributions, and public vs private split hints.
- Confirm how to format submission (per-base predictions melted to long with id_seqpos).

3) Validation Protocol:
- Use GroupKFold by sequence_id or stratification by sequence length/structure; mirror test: sequence-level CV, not token-level.
- Fix random seed; 5 folds. Save folds to disk.

4) Baseline Model (fast):
- Token features: sequence (A,C,G,U one-hot), structure (BPP features from provided pairing probability if present; else dot-bracket + simple pairing features), predicted_loop_type one-hot.
- Local context windows (k-mer embeddings) and position index features.
- Model A: Lightweight BiLSTM/GRU with attention, trained with MSE to match baseline (regression to targets); optionally multi-head outputs.
- Model B: CatBoost/XGBoost on pooled/contextual features (for quick sanity).

5) Feature Engineering v1:
- Base pairing probabilities (from bpps matrices if included or compute via ViennaRNA if allowed/time permits).
- Positional features: distance to paired base, GC content in window, k-mer counts, loop type runs.
- Temperature/Mg condition indicators per target head.

6) Training Details:
- Sequence-length bucketing and padding mask.
- Per-position loss masked to valid positions.
- Early stopping on CV fold OOF loss. Log progress, time/fold.
- Save OOF predictions and test predictions per seed.

7) Ensembling:
- Blend multiple seeds/architectures (BiLSTM + GRU; with/without BPP; small Transformer) by weighted average on OOF to pick weights, then apply to test.

8) Error Analysis:
- Analyze OOF by position bins, structure classes, sequence length, confidence; iterate on features.

9) Submission:
- Generate submission.csv exactly matching sample format.

Milestones (request expert review after each):
- Plan (this).
- Data audit + CV plan.
- Baseline training + OOF.
- FE v1 + improved model.
- Ensemble + final checks.

Next steps:
- 1) Verify GPU, install torch-cu121 stack.
- 2) Load and profile data; design CV folds.
- 3) Implement quick CatBoost baseline for smoke test and submission shape check, then move to BiLSTM.

In [1]:
# Environment check + data audit: GPU, schema, metric/target confirmation
import os, sys, json, time, shutil, subprocess
import pandas as pd, numpy as np
from pathlib import Path
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

def sh(cmd):
    print(f"$ {' '.join(cmd)}", flush=True)
    try:
        out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        print(out.stdout)
    except Exception as e:
        print('CMD failed:', e)

print('== GPU check ==')
sh(['bash','-lc','nvidia-smi || true'])

print('== Files ==')
print(os.listdir('.'))

print('== sample_submission.csv head/cols ==')
ss = pd.read_csv('sample_submission.csv')
print(ss.shape)
print(ss.head())
print('Columns:', ss.columns.tolist()[:20], '... total:', len(ss.columns))

print('== train.json schema/head ==')
try:
    tr = pd.read_json('train.json')
except ValueError:
    tr = pd.read_json('train.json', lines=True)
print(tr.shape)
print(tr.columns.tolist())
print(tr.head(2))
print(tr.dtypes)

# Inspect target nature and aux columns
cols = tr.columns.tolist()
targets = [c for c in cols if c.startswith('react') or c.startswith('deg')]
print('Target-like columns:', targets)
if targets:
    first = tr.iloc[0]
    for c in targets[:10]:
        v = first.get(c, None)
        if isinstance(v, (list, tuple, np.ndarray)) and len(v)>0:
            print(f'{c}: list len={len(v)}, type0={type(v[0])}')
        else:
            print(f'{c}: type={type(v)} value_sample={str(v)[:60]}')

for name in ['sequence','structure','predicted_loop_type','seq_scored','signal_to_noise','SN_filter']:
    print(f"Has {name}:", name in tr.columns)

# Sequence length distribution
if 'sequence' in tr.columns:
    tr['seq_len'] = tr['sequence'].astype(str).str.len()
    print('Train seq_len value_counts:\n', tr['seq_len'].value_counts().sort_index())
else:
    print('No sequence column found.')

print('== test.json schema/head ==')
try:
    te = pd.read_json('test.json')
except ValueError:
    te = pd.read_json('test.json', lines=True)
print(te.shape)
print(te.columns.tolist())
print(te.head(2))
if 'sequence' in te.columns:
    te['seq_len'] = te['sequence'].astype(str).str.len()
    print('Test seq_len value_counts:\n', te['seq_len'].value_counts().sort_index())

print('== sample_submission format inference ==')
print('First row id_seqpos:', ss.iloc[0,0])
print('Submission columns (first 10):', ss.columns[:10].tolist())


== GPU check ==
$ bash -lc nvidia-smi || true


Sat Sep 27 02:20:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Fast baseline: per-position means for 0..67; global means for 68..106; write submission.csv
import pandas as pd, numpy as np
from pathlib import Path

ss = pd.read_csv('sample_submission.csv')
try:
    tr = pd.read_json('train.json')
except ValueError:
    tr = pd.read_json('train.json', lines=True)
try:
    te = pd.read_json('test.json')
except ValueError:
    te = pd.read_json('test.json', lines=True)

targets = ['reactivity','deg_Mg_pH10','deg_pH10','deg_Mg_50C','deg_50C']

# Stack train targets into long dataframe with seqpos only for scored positions
rows = []
for _, r in tr.iterrows():
    scored = int(r['seq_scored'])
    for t in targets:
        vals = np.array(r[t], dtype=float)
        for i in range(scored):
            rows.append((t, i, vals[i]))
train_long = pd.DataFrame(rows, columns=['target','seqpos','value'])

# Compute per-position mean for 0..67 and global mean
pos_means = (train_long.groupby(['target','seqpos'])['value'].mean().unstack('seqpos'))
global_means = train_long.groupby('target')['value'].mean()
max_pos = pos_means.columns.max() if pos_means.shape[1] > 0 else 67
print('Computed pos_means up to position:', int(max_pos))

# Build prediction dict: for each test id and position 0..106
pred_map = {}  # key: (id, pos) -> dict of target->pred
for _, r in te[['id','seq_length','seq_scored']].iterrows():
    rid = r['id']
    L = int(r['seq_length'])
    scored = int(r['seq_scored'])  # expected 68
    for i in range(L):
        for t in targets:
            if i in pos_means.columns:
                val = float(pos_means.loc[t, i])
            else:
                # unscored positions fallback
                val = float(global_means.loc[t])
            pred_map[(rid, i, t)] = val

# Merge predictions into sample_submission to preserve order
def parse_id_seqpos(x):
    # 'id_xxx_pos'
    s, pos = x.rsplit('_', 1)
    return s, int(pos)

ids = []
pos = []
for v in ss['id_seqpos'].values:
    i, p = parse_id_seqpos(v)
    ids.append(i)
    pos.append(p)
ss['_id'] = ids
ss['_pos'] = pos

for t in targets:
    ss[t] = [pred_map[(i, p, t)] for i, p in zip(ss['_id'], ss['_pos'])]

ss.drop(columns=['_id','_pos'], inplace=True)
ss.to_csv('submission.csv', index=False)
print('Wrote submission.csv with shape:', ss.shape)
print(ss.head())

Computed pos_means up to position: 67


Wrote submission.csv with shape: (25680, 6)
        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00b436dec_0    0.531662     0.590755  2.001310    0.474894  0.608811
1  id_00b436dec_1    1.515921     2.078699  2.904483    2.025060  1.792079
2  id_00b436dec_2    1.117059     0.713412  0.918506    0.951581  1.013684
3  id_00b436dec_3    0.823220     0.540694  0.716204    0.728683  0.751078
4  id_00b436dec_4    0.664863     0.676643  0.731160    0.846463  0.736625


In [3]:
# Install PyTorch cu121 stack and sanity-check GPU
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any preinstalled torch stacks (idempotent)
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Freeze torch versions for later installs
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# 3) Sanity gate (hard fail on drift)
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 230.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 230.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 249.3 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 347.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 223.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 175.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 542.1 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q


In [5]:
# Build CV splits (StratifiedGroupKFold on SN_filter) and define masked MCRMSE metric
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.model_selection import GroupKFold
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold  # optional
except Exception:
    MultilabelStratifiedKFold = None
try:
    from sklearn.model_selection import StratifiedKFold
except Exception:
    StratifiedKFold = None

SEED = 42
N_FOLDS = 5
targets = ['reactivity','deg_Mg_pH10','deg_pH10','deg_Mg_50C','deg_50C']

def set_seed(seed=SEED):
    import random, os
    random.seed(seed); np.random.seed(seed)
set_seed()

# Robust JSON reader (handles non-line-delimited and line-delimited)
try:
    tr = pd.read_json('train.json')
except ValueError:
    tr = pd.read_json('train.json', lines=True)

# Grouping by sequence id; stratify by SN_filter (0/1).
groups = tr['id'].values
y_strat = tr['SN_filter'].values if 'SN_filter' in tr.columns else np.zeros(len(tr), dtype=int)

# Create folds
folds = np.full(len(tr), -1, dtype=int)
gkf = GroupKFold(n_splits=N_FOLDS)
splitter = gkf.split(np.zeros(len(tr)), y_strat, groups)
for fold, (tr_idx, va_idx) in enumerate(splitter):
    folds[va_idx] = fold

assert (folds >= 0).all(), 'Some folds not assigned'
tr_folds = tr[['id','SN_filter']].copy() if 'SN_filter' in tr.columns else tr[['id']].copy()
tr_folds['fold'] = folds
tr_folds.to_csv('folds.csv', index=False)
print('Saved folds.csv with shape:', tr_folds.shape)
print('Fold counts:\n', tr_folds['fold'].value_counts().sort_index())
if 'SN_filter' in tr_folds.columns:
    print('SN_filter by fold:\n', tr_folds.groupby('fold')['SN_filter'].value_counts().unstack(fill_value=0))

# Masked MCRMSE utility (evaluate on positions 0..67 only)
def masked_mcrmse(y_true, y_pred, mask):
    # y_* shape: [B, L, T], mask: [B, L] boolean for scored positions
    T = y_true.shape[-1]
    rmses = []
    for t in range(T):
        yt = y_true[..., t]; yp = y_pred[..., t]
        m = mask.astype(bool)
        diff = (yp - yt)[m]
        rmse = np.sqrt(np.mean(diff**2)) if diff.size > 0 else np.nan
        rmses.append(rmse)
    return float(np.nanmean(rmses))

print('Masked MCRMSE ready. Use mask = (pos < 68).')

Saved folds.csv with shape: (2160, 3)
Fold counts:
 fold
0    432
1    432
2    432
3    432
4    432
Name: count, dtype: int64
SN_filter by fold:
 SN_filter    0    1
fold               
0          154  278
1          149  283
2          170  262
3          168  264
4          170  262
Masked MCRMSE ready. Use mask = (pos < 68).


In [10]:
# Dataset, model, and training utilities with engineered structure features, standardization, SmoothL1 loss, Conv1D front-end
import math, random, gc, time, os
import numpy as np, pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)

# Encoders
BASES = ['A','C','G','U']
STRUCT = ['.', '(', ')']
LOOPS = list('BEHIMSX')
BASE2IDX = {b:i for i,b in enumerate(BASES)}
STRUCT2IDX = {c:i for i,c in enumerate(STRUCT)}
LOOP2IDX = {c:i for i,c in enumerate(LOOPS)}

def one_hot(idx, K):
    v = np.zeros(K, dtype=np.float32)
    if idx is not None and 0 <= idx < K: v[idx]=1.0
    return v

def dotbracket_partner_and_depth(struct):
    stack = []
    L = len(struct)
    partner = np.full(L, -1, dtype=np.int32)
    depth = np.zeros(L, dtype=np.int32)
    cur_depth = 0
    for i,ch in enumerate(struct):
        if ch == '(':
            stack.append(i); cur_depth += 1; depth[i] = cur_depth
        elif ch == ')':
            if stack:
                j = stack.pop();
                partner[i] = j; partner[j] = i
                depth[i] = cur_depth
            cur_depth = max(0, cur_depth-1)
        else:
            depth[i] = cur_depth
    return partner, depth

def run_lengths(arr):
    # length of the contiguous run containing each position (arr is array of codes/ints)
    L = len(arr); out = np.zeros(L, dtype=np.int32)
    start = 0
    for i in range(1, L+1):
        if i==L or arr[i] != arr[start]:
            out[start:i] = i - start
            start = i
    return out

def encode_row(row):
    seq = str(row['sequence']); struct = str(row['structure']); loop = str(row['predicted_loop_type'])
    L = len(seq)
    partner, depth = dotbracket_partner_and_depth(struct)
    is_paired = (partner >= 0).astype(np.float32)
    partner_dist = np.zeros(L, dtype=np.float32)
    for i in range(L):
        if partner[i] >= 0:
            partner_dist[i] = abs(partner[i] - i) / max(1,(L-1))
        else:
            partner_dist[i] = 0.0
    # stem run-length (paired runs) and loop run-length (same loop char) normalized
    paired_int = is_paired.astype(np.int32)
    stem_run = run_lengths(paired_int) / max(1, L)
    loop_codes = np.array([LOOP2IDX.get(c, 0) for c in loop], dtype=np.int32)
    loop_run = run_lengths(loop_codes) / max(1, L)
    # local GC ratio (win=5 centered)
    base_codes = np.array([BASE2IDX.get(b, -1) for b in seq], dtype=np.int32)
    is_gc = np.isin(base_codes, [BASE2IDX['G'], BASE2IDX['C']]).astype(np.float32)
    gc_win = np.zeros(L, dtype=np.float32)
    k = 5; rad = k//2
    csum = np.concatenate([[0.0], is_gc.cumsum()])
    for i in range(L):
        a = max(0, i - rad); b = min(L, i + rad + 1)
        gc_win[i] = (csum[b] - csum[a]) / max(1, b - a)
    feats = []
    for i,(b,s,l) in enumerate(zip(seq, struct, loop)):
        v = []
        v.extend(one_hot(BASE2IDX.get(b, -1), len(BASES)))
        v.extend(one_hot(STRUCT2IDX.get(s, -1), len(STRUCT)))
        v.extend(one_hot(LOOP2IDX.get(l, -1), len(LOOPS)))
        # positional scalars
        pos_norm = i / max(1,(L-1))
        v.append(float(i))
        v.append(float(L))
        v.append(pos_norm)
        v.append(math.sin(2*math.pi*pos_norm))
        v.append(math.cos(2*math.pi*pos_norm))
        # engineered continuous features
        v.append(float(is_paired[i]))
        v.append(float(partner_dist[i]))
        v.append(float(depth[i]))
        v.append(float(stem_run[i]))
        v.append(float(loop_run[i]))
        v.append(float(gc_win[i]))
        feats.append(v)
    x = np.asarray(feats, dtype=np.float32)  # [L, C]
    return x

TARGETS = ['reactivity','deg_Mg_pH10','deg_pH10','deg_Mg_50C','deg_50C']
SCORED_LEN = 68

def get_targets(row):
    y = np.zeros((len(str(row['sequence'])), len(TARGETS)), dtype=np.float32)
    for t_i, t in enumerate(TARGETS):
        arr = np.array(row[t], dtype=np.float32)[:SCORED_LEN]
        y[:SCORED_LEN, t_i] = arr
    return y

# Index where continuous channels start (one-hots first): 4 + 3 + 7 = 14
CONT_START = 4 + 3 + 7

class TrainDataset(Dataset):
    def __init__(self, df, scaler=None, sample_weights=None):
        self.df = df.reset_index(drop=True)
        self.scaler = scaler  # dict with 'mean','std' for continuous cols
        self.sample_weights = sample_weights if sample_weights is not None else np.ones(len(self.df), dtype=np.float32)
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        x = encode_row(r)  # [L, C]
        if self.scaler is not None:
            mu = self.scaler['mean']; sd = self.scaler['std']
            x_cont = x[:, CONT_START:]
            x[:, CONT_START:] = np.clip((x_cont - mu) / (sd + 1e-6), -5.0, 5.0)
        y = get_targets(r) # [L, T]
        L = x.shape[0]
        mask = np.zeros((L,), dtype=np.float32)
        mask[:SCORED_LEN]=1.0
        w = float(self.sample_weights[idx])
        return x, y, mask, r['id'], w

class TestDataset(Dataset):
    def __init__(self, df, scaler=None):
        self.df = df.reset_index(drop=True); self.scaler = scaler
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        x = encode_row(r)
        if self.scaler is not None:
            mu = self.scaler['mean']; sd = self.scaler['std']
            x_cont = x[:, CONT_START:]
            x[:, CONT_START:] = np.clip((x_cont - mu) / (sd + 1e-6), -5.0, 5.0)
        L = x.shape[0]
        mask = np.zeros((L,), dtype=np.float32)
        mask[:SCORED_LEN]=1.0
        return x, mask, r['id']

def collate_train(batch):
    lens = [b[0].shape[0] for b in batch]
    maxL = max(lens)
    C = batch[0][0].shape[1]
    T = batch[0][1].shape[1]
    B = len(batch)
    x = np.zeros((B,maxL,C), dtype=np.float32)
    y = np.zeros((B,maxL,T), dtype=np.float32)
    mask = np.zeros((B,maxL), dtype=np.float32)
    ids = []; w = np.zeros((B,), dtype=np.float32)
    for i,(xi, yi, mi, idv, wi) in enumerate(batch):
        L = xi.shape[0]
        x[i,:L,:] = xi
        y[i,:L,:] = yi
        mask[i,:L] = mi
        ids.append(idv); w[i]=wi
    return torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(mask), ids, torch.from_numpy(w)

def collate_test(batch):
    lens = [b[0].shape[0] for b in batch]
    maxL = max(lens); C = batch[0][0].shape[1]; B=len(batch)
    x = np.zeros((B,maxL,C), dtype=np.float32)
    mask = np.zeros((B,maxL), dtype=np.float32)
    ids = []
    for i,(xi, mi, idv) in enumerate(batch):
        L = xi.shape[0]
        x[i,:L,:]=xi; mask[i,:L]=mi; ids.append(idv)
    return torch.from_numpy(x), torch.from_numpy(mask), ids

class CNNBiLSTMModel(nn.Module):
    def __init__(self, in_ch, conv_ch=128, hidden=256, num_layers=2, out_ch=5, dropout=0.3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, conv_ch, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.lstm = nn.LSTM(input_size=conv_ch, hidden_size=hidden, num_layers=num_layers,
                            dropout=dropout if num_layers>1 else 0.0, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.head = nn.Linear(hidden*2, out_ch)
    def forward(self, x, lengths):
        # x: [B,L,C]; lengths: [B]
        x = x.transpose(1,2)  # [B,C,L]
        x = self.conv(x)
        x = x.transpose(1,2)  # [B,L,conv_ch]
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out_packed, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(out_packed, batch_first=True)
        out = self.dropout(out)
        y = self.head(out)  # [B,L,5]
        return y

def _lengths_from_x(xb):
    return (xb.abs().sum(dim=2) > 0).sum(dim=1).long().clamp(min=1)

def masked_smoothl1_loss(pred, target, mask, sample_weights=None, beta=1.0):
    # pred/target: [B,L,T], mask: [B,L], sample_weights: [B]
    loss = F.smooth_l1_loss(pred, target, reduction='none', beta=beta)  # [B,L,T]
    mask3 = mask.unsqueeze(-1)
    loss = loss * mask3
    denom = (mask3.sum(dim=(1,2)).clamp(min=1.0))  # [B]
    loss_b = loss.sum(dim=(1,2)) / denom  # [B]
    if sample_weights is not None:
        loss_b = loss_b * sample_weights
        return loss_b.mean()
    else:
        return loss_b.mean()

def compute_scaler(train_df):
    # Fit mean/std for continuous channels from train fold only
    xs = []
    for _, r in train_df.iterrows():
        x = encode_row(r)
        xs.append(x[:, CONT_START:])
    X = np.concatenate(xs, axis=0)  # [sumL, C_cont]
    mu = X.mean(axis=0).astype(np.float32)
    sd = X.std(axis=0).astype(np.float32)
    return {'mean': mu, 'std': sd}

def run_fold(fold, tr_df, te_df, folds_df, epochs=20, batch_size=64, lr=1e-3, wd=1e-4, use_amp=True):
    # split ids
    tr_ids = set(tr_df['id'].values)
    train_ids = set(folds_df.loc[folds_df['fold']!=fold, 'id'].values) & tr_ids
    valid_ids = set(folds_df.loc[folds_df['fold']==fold, 'id'].values) & tr_ids
    dtr = tr_df[tr_df['id'].isin(train_ids)].reset_index(drop=True)
    dva = tr_df[tr_df['id'].isin(valid_ids)].reset_index(drop=True)
    # sample weights by SN_filter
    if 'SN_filter' in dtr.columns:
        sw_tr = np.where(dtr['SN_filter'].values.astype(int)==1, 1.0, 0.5).astype(np.float32)
    else:
        sw_tr = np.ones(len(dtr), dtype=np.float32)
    # scaler per fold
    scaler = compute_scaler(dtr)
    ds_tr = TrainDataset(dtr, scaler=scaler, sample_weights=sw_tr)
    ds_va = TrainDataset(dva, scaler=scaler, sample_weights=np.ones(len(dva), dtype=np.float32))
    ds_te = TestDataset(te_df, scaler=scaler)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, collate_fn=collate_train)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, collate_fn=collate_train)
    dl_te = DataLoader(ds_te, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, collate_fn=collate_test)
    in_ch = ds_tr[0][0].shape[1]
    model = CNNBiLSTMModel(in_ch=in_ch).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=2, factor=0.5, verbose=False)
    scaler_amp = torch.cuda.amp.GradScaler(enabled=use_amp and DEVICE=='cuda')
    best = 1e9; best_state=None; start=time.time()
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0.0; n_batches=0; t0=time.time()
        for xb, yb, mb, _, wb in dl_tr:
            xb=xb.to(DEVICE); yb=yb.to(DEVICE); mb=mb.to(DEVICE); wb=wb.to(DEVICE); lens = _lengths_from_x(xb)
            opt.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda', enabled=use_amp and DEVICE=='cuda'):
                preds = model(xb, lens)
                loss = masked_smoothl1_loss(preds, yb, mb, sample_weights=wb, beta=1.0)
            scaler_amp.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler_amp.step(opt); scaler_amp.update()
            tr_loss += loss.item(); n_batches+=1
        # validate
        model.eval(); ys=[]; ps=[]; ms=[]
        with torch.no_grad():
            for xb, yb, mb, _, _ in dl_va:
                xb=xb.to(DEVICE); yb=yb.to(DEVICE); mb=mb.to(DEVICE); lens = _lengths_from_x(xb)
                preds = model(xb, lens)
                ys.append(yb.cpu().numpy()); ps.append(preds.cpu().numpy()); ms.append(mb.cpu().numpy())
        y_true = np.concatenate(ys, axis=0); y_pred = np.concatenate(ps, axis=0); m = np.concatenate(ms, axis=0).astype(bool)
        # MCRMSE
        rmses=[]
        for t in range(y_true.shape[-1]):
            diff = (y_pred[...,t]-y_true[...,t])[m]
            rmse = float(np.sqrt(np.mean(diff**2))) if diff.size>0 else np.nan
            rmses.append(rmse)
        mcrmse = float(np.nanmean(rmses))
        scheduler.step(mcrmse)
        per_target_str = '[' + ', '.join(str(round(r,5)) for r in rmses) + ']'
        print(f'Fold {fold} Epoch {ep}/{epochs} tr_loss={tr_loss/max(1,n_batches):.5f} val_MCRMSE={mcrmse:.5f} per-target={per_target_str} time_ep={time.time()-t0:.1f}s elapsed={time.time()-start:.1f}s', flush=True)
        if mcrmse < best: best=mcrmse; best_state = {k:v.cpu() for k,v in model.state_dict().items()}
    if best_state is not None: model.load_state_dict(best_state)
    # OOF preds
    model.eval(); ys=[]; ps=[]; ms=[]; va_ids=[]
    with torch.no_grad():
        for xb, yb, mb, ids, _ in dl_va:
            xb=xb.to(DEVICE); mb=mb.to(DEVICE); lens = _lengths_from_x(xb)
            preds = model(xb, lens)
            ys.append(yb.cpu().numpy()); ps.append(preds.cpu().numpy()); ms.append(mb.cpu().numpy()); va_ids.extend(ids)
    oof_y = np.concatenate(ys, axis=0); oof_p = np.concatenate(ps, axis=0); oof_m = np.concatenate(ms, axis=0).astype(bool)
    # Test preds with reverse TTA
    test_preds = [] ; test_ids = []
    with torch.no_grad():
        for xb, mb, ids in dl_te:
            xb=xb.to(DEVICE); mb=mb.to(DEVICE); lens = _lengths_from_x(xb)
            preds_fwd = model(xb, lens)
            xb_rev = xb.flip(1); lens_rev = lens  # same lengths
            preds_rev = model(xb_rev, lens_rev).flip(1)
            preds = 0.5*(preds_fwd + preds_rev)
            test_preds.append(preds.cpu().numpy()); test_ids.extend(ids)
    test_pred = np.concatenate(test_preds, axis=0)
    return oof_y, oof_p, oof_m, va_ids, test_pred, test_ids, best

print('Dataset/model utilities ready (engineered features + standardization + SmoothL1 + Conv1D + reverse TTA).')

Dataset/model utilities ready (engineered features + standardization + SmoothL1 + Conv1D + reverse TTA).


In [11]:
# Orchestrate 5-fold training, aggregate OOF and test preds, and build submission
import pandas as pd, numpy as np, time, os

# Load data and folds
try:
    tr = pd.read_json('train.json')
except ValueError:
    tr = pd.read_json('train.json', lines=True)
try:
    te = pd.read_json('test.json')
except ValueError:
    te = pd.read_json('test.json', lines=True)
folds_df = pd.read_csv('folds.csv')

N_FOLDS = 5
EPOCHS = 20  # increased for stronger training with engineered features
BATCH_SIZE = 64
LR = 1e-3
WD = 1e-4

all_oof_true = []
all_oof_pred = []
all_oof_mask = []
all_oof_ids = []
test_pred_accum = None
test_ids_ref = None
fold_scores = []

t0 = time.time()
for fold in range(N_FOLDS):
    print(f'===== Fold {fold} / {N_FOLDS} =====', flush=True)
    oof_y, oof_p, oof_m, va_ids, te_pred, te_ids, best = run_fold(
        fold, tr, te, folds_df, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR, wd=WD, use_amp=True
    )
    # compute fold MCRMSE
    m = oof_m.astype(bool)
    rmses=[]
    for t in range(oof_y.shape[-1]):
        diff = (oof_p[...,t]-oof_y[...,t])[m]
        rmse = float(np.sqrt(np.mean(diff**2))) if diff.size>0 else np.nan
        rmses.append(rmse)
    mcrmse = float(np.nanmean(rmses))
    fold_scores.append(mcrmse)
    print(f'Fold {fold} best_val_MCRMSE={best:.5f} OOF_MCRMSE={mcrmse:.5f} per-target={[round(r,5) for r in rmses]}', flush=True)
    all_oof_true.append(oof_y); all_oof_pred.append(oof_p); all_oof_mask.append(oof_m); all_oof_ids.extend(va_ids)
    # accumulate test preds (align by te_ids order)
    if test_pred_accum is None:
        test_pred_accum = te_pred.copy()
        test_ids_ref = te_ids
    else:
        # ensure same order
        assert test_ids_ref == te_ids, 'Test id order mismatch across folds'
        test_pred_accum += te_pred

print('Fold scores:', fold_scores, 'mean:', float(np.nanmean(fold_scores)))

# Average test predictions across folds
test_pred_mean = test_pred_accum / N_FOLDS
print('Test pred shape:', test_pred_mean.shape)

# Build submission by merging onto sample_submission order
ss = pd.read_csv('sample_submission.csv')

# Map test id to row index in te dataframe to access its prediction row
id_to_idx = {idv:i for i, idv in enumerate(test_ids_ref)}
targets = ['reactivity','deg_Mg_pH10','deg_pH10','deg_Mg_50C','deg_50C']

def parse_id_seqpos(x):
    s, pos = x.rsplit('_', 1)
    return s, int(pos)

ids = []
pos = []
for v in ss['id_seqpos'].values:
    i, p = parse_id_seqpos(v)
    ids.append(i); pos.append(p)
ss['_id'] = ids; ss['_pos'] = pos

for t_i, t in enumerate(targets):
    vals = []
    for i, p in zip(ss['_id'].values, ss['_pos'].values):
        idx = id_to_idx[i]
        vals.append(float(test_pred_mean[idx, p, t_i]))
    ss[t] = vals

ss.drop(columns=['_id','_pos'], inplace=True)
ss.to_csv('submission.csv', index=False)
print('Wrote submission.csv with shape:', ss.shape)

# Compute overall OOF MCRMSE
oof_y_full = np.concatenate(all_oof_true, axis=0)
oof_p_full = np.concatenate(all_oof_pred, axis=0)
oof_m_full = np.concatenate(all_oof_mask, axis=0).astype(bool)
rmses=[]
for t in range(oof_y_full.shape[-1]):
    diff = (oof_p_full[...,t]-oof_y_full[...,t])[oof_m_full]
    rmse = float(np.sqrt(np.mean(diff**2))) if diff.size>0 else np.nan
    rmses.append(rmse)
mcrmse = float(np.nanmean(rmses))
print(f'OOF MCRMSE={mcrmse:.5f} per-target={[round(r,5) for r in rmses]} total_time={time.time()-t0:.1f}s')

print('Done. Next: consider adding BPP features and reverse TTA to improve scores.')

===== Fold 0 / 5 =====


/app/.pip-target/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipykernel_196/502443009.py:257: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_amp = torch.cuda.amp.GradScaler(enabled=use_amp and DEVICE=='cuda')


Fold 0 Epoch 1/20 tr_loss=0.51085 val_MCRMSE=0.92862 per-target=[0.67074, 0.69005, 1.28625, 0.88451, 1.11155] time_ep=1.7s elapsed=1.7s


Fold 0 Epoch 2/20 tr_loss=0.40382 val_MCRMSE=0.91585 per-target=[0.65898, 0.67288, 1.26874, 0.87228, 1.10636] time_ep=1.6s elapsed=3.3s


Fold 0 Epoch 3/20 tr_loss=0.38127 val_MCRMSE=0.90645 per-target=[0.65106, 0.65795, 1.25803, 0.86288, 1.10235] time_ep=1.6s elapsed=4.9s


...

Fold 4 Epoch 18/20 tr_loss=0.30794 val_MCRMSE=0.91841 per-target=[0.70403, 0.61184, 1.29845, 0.83202, 1.14571] time_ep=1.7s elapsed=29.7s


Fold 4 Epoch 19/20 tr_loss=0.30789 val_MCRMSE=0.91808 per-target=[0.70334, 0.61112, 1.29839, 0.83162, 1.14593] time_ep=1.6s elapsed=31.4s


Fold 4 Epoch 20/20 tr_loss=0.30603 val_MCRMSE=0.91724 per-target=[0.70284, 0.61073, 1.29717, 0.83062, 1.14483] time_ep=1.7s elapsed=33.0s


Fold 4 best_val_MCRMSE=0.91724 OOF_MCRMSE=0.91724 per-target=[0.70284, 0.61073, 1.29717, 0.83062, 1.14483]


Fold scores: [0.8775848388671875, 0.8532713294029236, 0.8385927438735962, 0.976087236404419, 0.917236077785492] mean: 0.8925544452667236
Test pred shape: (240, 107, 5)
Wrote submission.csv with shape: (25680, 6)
OOF MCRMSE=0.89440 per-target=[0.6665, 0.61091, 1.25845, 0.80975, 1.12639] total_time=179.1s
Done. Next: consider adding BPP features and reverse TTA to improve scores.


In [9]:
# Install ViennaRNA (RNAplfold), compute BPP features for all ids, and cache per-position aggregates
import os, sys, time, math, json, shutil, subprocess
import numpy as np, pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

def sh(cmd):
    print(f"$ {cmd}", flush=True)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(p.stdout)
    return p.returncode

start = time.time()
print('== Install ViennaRNA (RNAplfold) ==', flush=True)
rc = sh('apt-get update -qq && apt-get install -y -qq viennarna')
if rc != 0:
    print('APT install failed; trying pip ViennaRNAx (fallback).', flush=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ViennaRNAx'], check=True)
print('RNAplfold version/help:')
sh('RNAplfold -h || true')

# Load train/test and collect sequences
try:
    tr = pd.read_json('train.json')
except ValueError:
    tr = pd.read_json('train.json', lines=True)
try:
    te = pd.read_json('test.json')
except ValueError:
    te = pd.read_json('test.json', lines=True)

all_df = pd.concat([tr[['id','sequence']], te[['id','sequence']]], ignore_index=True)
ids = all_df['id'].tolist()
seqs = all_df['sequence'].tolist()

work_root = Path('plfold_work'); work_root.mkdir(exist_ok=True)
out_root = Path('plfold_out'); out_root.mkdir(exist_ok=True)
feat_root = Path('bpp_features'); feat_root.mkdir(exist_ok=True)

def run_plfold(one):
    idv, seq = one
    d = work_root / idv
    out_dp = out_root / f'{idv}_dp.ps'
    out_lunp = out_root / f'{idv}_lunp'
    if out_dp.exists() and out_lunp.exists():
        return idv, True
    if d.exists():
        shutil.rmtree(d, ignore_errors=True)
    d.mkdir(parents=True, exist_ok=True)
    fasta = d / f'{idv}.fa'
    fasta.write_text(f'>{idv}\n{seq}\n')
    env = os.environ.copy()
    env['OMP_NUM_THREADS'] = '1'
    cmd = f'RNAplfold -W 150 -L 120 -u 3 -noLP < {fasta.name}'
    p = subprocess.run(cmd, shell=True, cwd=str(d), env=env, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if p.returncode != 0:
        return idv, False
    # Move outputs
    dp_src = d / 'plfold_dp.ps'
    lunp_src = d / 'plfold_lunp'
    if dp_src.exists():
        shutil.move(str(dp_src), str(out_dp))
    if lunp_src.exists():
        shutil.move(str(lunp_src), str(out_lunp))
    shutil.rmtree(d, ignore_errors=True)
    return idv, (out_dp.exists() and out_lunp.exists())

print('== Running RNAplfold in parallel ==', flush=True)
ok = 0; fail = 0; t0 = time.time()
with ThreadPoolExecutor(max_workers=min(16, os.cpu_count() or 8)) as ex:
    futures = {ex.submit(run_plfold, item): item[0] for item in zip(ids, seqs)}
    for i, fut in enumerate(as_completed(futures), 1):
        idv = futures[fut]
        try:
            _, success = fut.result()
            if success: ok += 1
            else: fail += 1
        except Exception:
            fail += 1
        if i % 100 == 0 or i == len(futures):
            print(f'Processed {i}/{len(futures)} ok={ok} fail={fail} elapsed={time.time()-t0:.1f}s', flush=True)

def parse_lunp(path):
    # Returns pu1 (unpaired prob for u=1) per position, length L
    lines = Path(path).read_text().strip().splitlines()
    vals = []
    # Try two formats: with leading position index or not
    for ln in lines:
        parts = ln.strip().split()
        if not parts: continue
        # Expect at least pu1 present
        if len(parts) == 1:
            # Single column unlikely; treat as pu1
            vals.append(float(parts[0]))
        else:
            # If first token is integer index
            try:
                int(parts[0]); has_idx = True
            except ValueError:
                has_idx = False
            pu1 = float(parts[1] if has_idx else parts[0])
            vals.append(pu1)
    return np.array(vals, dtype=np.float32)

def parse_dp_ps(path, L):
    # Build LxL probability matrix from _dp.ps; lines like: i j p_sqrt ubox
    P = np.zeros((L, L), dtype=np.float32)
    for ln in Path(path).read_text().splitlines():
        ln = ln.strip()
        if not ln or ln.startswith('%'):
            continue
        parts = ln.split()
        if len(parts) >= 4 and parts[-1] == 'ubox':
            try:
                i = int(parts[0]) - 1
                j = int(parts[1]) - 1
                psqrt = float(parts[2])
            except Exception:
                continue
            p = psqrt * psqrt
            if 0 <= i < L and 0 <= j < L:
                P[i, j] = max(P[i, j], p)
                P[j, i] = max(P[j, i], p)
    return P

def build_features(P, pu1):
    L = P.shape[0]
    eps = 1e-8
    row_sum = P.sum(axis=1)  # [L]
    row_max = P.max(axis=1)
    # entropy on normalized rows
    Q = P / (row_sum[:, None] + eps)
    entropy = -(Q * (np.log(Q + eps))).sum(axis=1)
    # expected distance normalized
    idx = np.arange(L, dtype=np.float32)
    D = np.abs(idx[None, :] - idx[:, None])
    exp_dist = (P * D).sum(axis=1) / (row_sum + eps) / max(1.0, (L - 1))
    # local mass windows
    feats = [row_sum, row_max, entropy, exp_dist]
    for k in (3, 7, 15):
        mask = (D <= k).astype(np.float32)
        lm = (P * mask).sum(axis=1)
        feats.append(lm)
    # append pu1 and pairedness
    pu1 = pu1.astype(np.float32)
    if pu1.shape[0] != L:
        # pad or trim
        pu1_fix = np.zeros((L,), dtype=np.float32)
        m = min(L, pu1.shape[0])
        pu1_fix[:m] = pu1[:m]
        pu1 = pu1_fix
    paired = 1.0 - pu1
    feats.append(pu1); feats.append(paired)
    X = np.stack(feats, axis=1).astype(np.float32)  # [L, C]
    return X

print('== Parsing RNAplfold outputs and caching features ==', flush=True)
n_done = 0
for idv, seq in zip(ids, seqs):
    npy_out = feat_root / f'{idv}.npy'
    if npy_out.exists():
        n_done += 1; continue
    dp = out_root / f'{idv}_dp.ps'
    lunp = out_root / f'{idv}_lunp'
    if not (dp.exists() and lunp.exists()):
        # missing outputs; write zeros as fallback
        L = len(seq)
        np.save(npy_out, np.zeros((L, 10), dtype=np.float32))
        continue
    L = len(seq)
    P = parse_dp_ps(dp, L)
    pu1 = parse_lunp(lunp)
    X = build_features(P, pu1)  # [L, ~10]
    np.save(npy_out, X)
    n_done += 1
    if n_done % 200 == 0:
        print(f'Cached {n_done}/{len(ids)} feature arrays...', flush=True)

print(f'Done BPP features. Cached {n_done} arrays. Total time {time.time()-start:.1f}s', flush=True)

== Install ViennaRNA (RNAplfold) ==


$ apt-get update -qq && apt-get install -y -qq viennarna


E: List directory /var/lib/apt/lists/partial is missing. - Acquire (30: Read-only file system)

APT install failed; trying pip ViennaRNAx (fallback).


ERROR: Could not find a version that satisfies the requirement ViennaRNAx (from versions: none)
ERROR: No matching distribution found for ViennaRNAx


CalledProcessError: Command '['/usr/bin/python3.11', '-m', 'pip', 'install', 'ViennaRNAx']' returned non-zero exit status 1.